In [4]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json

In [5]:
r = requests.get('https://www.themalaysianinsight.com/chinese')

In [6]:
soup = BeautifulSoup(r.content, "lxml")
a = soup.find_all('a')
a = [a_.get('href') for a_ in a if a_.get('href')]

In [7]:
topics = list(set([a_ for a_ in a if 'https://www.themalaysianinsight.com/chinese/c' in a_]))
topics

['https://www.themalaysianinsight.com/chinese/c/most-read',
 'https://www.themalaysianinsight.com/chinese/c/notices',
 'https://www.themalaysianinsight.com/chinese/c/other-news',
 'https://www.themalaysianinsight.com/chinese/c/pictures',
 'https://www.themalaysianinsight.com/chinese/c/editorial',
 'https://www.themalaysianinsight.com/chinese/c/your-insight',
 'https://www.themalaysianinsight.com/chinese/c/voices',
 'https://www.themalaysianinsight.com/chinese/c/videos',
 'https://www.themalaysianinsight.com/chinese/c/news']

In [9]:
topic_c = '-'.join(topics[0].split('/')[-3:])
topic_c

'chinese-c-most-read'

In [15]:
def crawl(url, filename):
    if os.path.exists(filename):
        return True
    while True:
        try:
            r = requests.get(url)
            break
        except:
            time.sleep(1.0)
    soup = BeautifulSoup(r.content, "lxml")
    divs = soup.find_all('div', {'class': 'story'})
    urls = []
    for div in divs:
        a = div.find_all('a')
        a = [a_.get('href') for a_ in a if a_.get('href')]
        a = list(set([a_ for a_ in a if 'https://www.themalaysianinsight.com/' in a_]))
        urls.extend(a)
    with open(filename, 'w') as fopen:
        json.dump(urls, fopen)
    return len(urls) > 0

In [16]:
directory = 'url'
!mkdir {directory}

mkdir: cannot create directory ‘url’: File exists


In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed

max_worker = 10

for topic in topics:
    if 'most-read' in topic:
        continue
    topic_c = '-'.join(topic.split('/')[-3:])
    print(topic_c)
    done = False
    for i in tqdm(range(0, 100000, max_worker)):
        aranged = np.arange(i, i + max_worker)
        urls = [(f'{topic}?page={a}', 
                 os.path.join(directory, f'{topic_c}-{a}.json')) for a in aranged]
        with ThreadPoolExecutor(max_workers=max_worker) as executor:
            futures = {executor.submit(crawl, url[0], url[1]): url for url in urls}
        
        
        for future in as_completed(futures):
            if not future.result():
                done = True
        
        if done:
            break

chinese-c-notices


  0%|                                                 | 0/10000 [00:20<?, ?it/s]


chinese-c-other-news


  0%|                                     | 12/10000 [03:08<43:32:12, 15.69s/it]


chinese-c-pictures


...

  0%|                                      | 2/10000 [00:43<59:58:32, 21.60s/it]


chinese-c-your-insight


  0%|                                      | 1/10000 [00:29<81:08:04, 29.21s/it]


chinese-c-voices


  2%|▊                                 | 227/10000 [1:11:31<18:23:28,  6.77s/it]